In [1]:
import os, sys
import pandas as pd
import numpy as np
import json

In [2]:
# path 설정
sys.path.append((os.path.abspath("")))
print(sys.path[-1])

d:\base\boostcamp\temp\code


In [3]:
data_path: str = "data/eval/ens"
ease: pd.DataFrame = pd.read_csv(os.path.join(data_path,"submission-ease.csv"))
recvae: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recvae_100_score.csv"))
bert: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_bert4rec_top20_score.csv"))
lgcn: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_lightgcn_1126_top20_score.csv"))
admm: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_ADMMSLIM_100_top20_score.csv"))
autoint: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_autoint_top20_score.csv"))
bpr: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_bpr_top20_score.csv"))
gru: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_gru4recf_top20_score.csv"))


In [4]:
# 각 데이터프레임에 대해 유저별 순위 매기기
ease['rank'] = ease.groupby('user').cumcount() + 1
recvae['rank'] = recvae.groupby('user').cumcount() + 1  
bert['rank'] = bert.groupby('user').cumcount() + 1
lgcn['rank'] = lgcn.groupby('user').cumcount() + 1
admm['rank'] = admm.groupby('user').cumcount() + 1
autoint['rank'] = autoint.groupby('user').cumcount() + 1
bpr['rank'] = bpr.groupby('user').cumcount() + 1
gru['rank'] = gru.groupby('user').cumcount() + 1


In [5]:
### 순위 합산

import pandas as pd
from collections import defaultdict
from tqdm import tqdm

dataframe_list = [ease, admm, gru, recvae, lgcn]
user_list = dataframe_list[0]['user'].unique()
result = []
tbar = tqdm(user_list, desc='Rank Aggregation with Normalization')
for user in tbar:
    rank_sum = defaultdict(float)
    for df_idx, df in enumerate(dataframe_list):
        items = df[df['user'] == user]['item'].values
        max_rank = min(len(items), 20)
        for rank_idx in range(max_rank):
            # 순위를 정규화하여 합산 (1위는 1, 20위는 0.05)
            normalized_rank = (max_rank - rank_idx) / max_rank
            rank_sum[items[rank_idx]] += normalized_rank
    # 순위의 합이 낮을수록 좋은 아이템 선택
    top_items = sorted(rank_sum.items(), key=lambda x: x[1], reverse=True)[:10]
    for item, _ in top_items:
        result.append((user, item))
submission = pd.DataFrame(result, columns=['user', 'item'])

Rank Aggregation with Normalization: 100%|██████████| 31360/31360 [01:32<00:00, 338.16it/s]


In [ ]:
# ### 등장횟수 합산


# # 각 유저별로 아이템 등장 횟수를 계산
# result = []
# user_list = ease['user'].unique()
# tbar = tqdm(user_list, desc='Rank Aggregation with Count')

# for user in tbar:
#     item_count = defaultdict(int)
    
#     # ease와 admm 각각에서 해당 유저의 아이템 카운트
#     for df in [ease, admm]:
#         user_items = df[df['user'] == user]['item'].values
#         for item in user_items[:20]:  # 상위 20개 아이템만 고려
#             item_count[item] += 1
            
#     # 카운트가 높은 순으로 정렬하여 상위 10개 아이템 선택
#     top_items = sorted(item_count.items(), key=lambda x: x[1], reverse=True)[:10]
    
#     # 결과에 추가
#     for item, _ in top_items:
#         result.append((user, item))

# submission = pd.DataFrame(result, columns=['user', 'item'])


In [6]:
submission


,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,32587
4,11,40815
...,...,...
313595,138493,48394
313596,138493,8961
313597,138493,5349
313598,138493,53125


In [7]:
sub_path = "data/eval/ens/submission"
submission.to_csv(os.path.join(sub_path,"ens_hard_esgrl.csv"), index=False)